In [ ]:
!pip install pandas

In [1]:
import pandas as pd

In [23]:
df_mov_taz_uber = pd.read_csv("../data/mov_taz_ids_sf_n_o_minimal.csv")

In [24]:
df_taz_edge_sf = pd.read_csv("../data/TAZ_edge.csv")
df_taz_edge_sf.taz_id = df_taz_edge_sf.taz_id.astype(str)
df_taz_edge_sf.edge_id = df_taz_edge_sf.edge_id.astype(str)

In [25]:
df_taz_edge_sf.head()

,taz_id,edge_id
0,237,-120257662
1,237,-120257665
2,237,-133856290
3,237,-133856291
4,237,-215968721#1


In [26]:
df_taz_edge_uber = pd.read_csv("../data/sf_n_o_minimal_edge.csv")
df_taz_edge_uber.taz_id = df_taz_edge_uber.taz_id.astype(str)
df_taz_edge_uber.edge_id = df_taz_edge_uber.edge_id.astype(str)

In [27]:
df_trip_taz_sf = pd.read_csv("../data/trip_stats_taz_sf.csv")
df_trip_taz_sf.taz_id = df_trip_taz_sf.taz_id.astype(str)

In [28]:
df_trip_taz_sf.head()

,taz_id,day_of_week,hour,pickups,dropoffs
0,1,0,3,0.6,0.4
1,1,0,4,0.0,0.4
2,1,0,5,0.6,0.6
3,1,0,6,0.8,1.0
4,1,0,7,0.6,0.8


In [29]:
df_mov_edge_uber = pd.merge(df_mov_taz_uber, df_taz_edge_uber, how ='inner', on =["taz_id"])[["movement_id","edge_id"]]
df_mov_edge_uber.edge_id = df_mov_edge_uber.edge_id.astype(str)

In [30]:
df_trip_taz_mean_gb_hour = df_trip_taz_sf.groupby(["taz_id","hour"]).mean().reset_index()[["taz_id","hour","pickups","dropoffs"]]

In [31]:
df_trip_taz_mean_gb_hour.head()

,taz_id,hour,pickups,dropoffs
0,1,3,0.121429,0.235714
1,1,4,0.142857,0.157143
2,1,5,0.521429,0.678571
3,1,6,0.600000,0.621429
4,1,7,0.635714,0.678571


In [32]:
df_taz_edge_pickups_dropoffs_sf = pd.merge(df_trip_taz_mean_gb_hour, df_taz_edge_sf, how ='inner', on =["taz_id"])

In [33]:
df_taz_edge_pickups_dropoffs_sf.head()

,taz_id,hour,pickups,dropoffs,edge_id
0,237,3,1.614286,1.55,-120257662
1,237,3,1.614286,1.55,-120257665
2,237,3,1.614286,1.55,-133856290
3,237,3,1.614286,1.55,-133856291
4,237,3,1.614286,1.55,-215968721#1


In [34]:
df_taz_edge_pickups_dropoffs_sf.shape

(264576, 5)

In [35]:
df_edge_mov_pick_drop_uber = pd.merge(df_mov_edge_uber, df_taz_edge_pickups_dropoffs_sf, how="inner", on=["edge_id"])

In [36]:
df_edge_mov_pick_drop_uber.head()

,movement_id,edge_id,taz_id,hour,pickups,dropoffs
0,3394,-143666206#0,852,3,5.385714,4.907143
1,3394,-143666206#0,852,4,7.928571,5.692857
2,3394,-143666206#0,852,5,12.500000,6.728571
3,3394,-143666206#0,852,6,18.942857,12.371429
4,3394,-143666206#0,852,7,30.900000,21.428571


In [37]:
df_mov_mean_pick_drop = df_edge_mov_pick_drop_uber.groupby(["movement_id","hour"]).mean().reset_index()[["movement_id","hour","pickups","dropoffs"]]
df_mov_mean_pick_drop.columns = ["movement_id","hour","pickups_mean","dropoffs_mean"]

In [38]:
df_mov_std_pick_drop = df_edge_mov_pick_drop_uber.groupby(["movement_id","hour"]).std().reset_index()[["movement_id","hour","pickups","dropoffs"]]
df_mov_std_pick_drop.columns = ["movement_id","hour","pickups_std","dropoffs_std"]

In [39]:
df_mov_mean_pick_drop.head()

,movement_id,hour,pickups_mean,dropoffs_mean
0,3394,3,4.660534,4.234055
1,3394,4,6.785281,4.896392
2,3394,5,11.012626,5.970491
3,3394,6,16.979798,10.966306
4,3394,7,28.943795,19.915152


In [40]:
df_mov_std_pick_drop.head()

,movement_id,hour,pickups_std,dropoffs_std
0,3394,3,1.495611,1.388176
1,3394,4,2.357921,1.642628
2,3394,5,3.067559,1.563465
3,3394,6,4.048612,2.897924
4,3394,7,4.034476,3.121276


In [41]:
df_mov_mean_std_pick_drop = pd.merge(df_mov_mean_pick_drop, df_mov_std_pick_drop, how="inner", on=["movement_id","hour"]).reset_index()

In [42]:
df_mov_mean_std_pick_drop.head()

,index,movement_id,hour,pickups_mean,dropoffs_mean,pickups_std,dropoffs_std
0,0,3394,3,4.660534,4.234055,1.495611,1.388176
1,1,3394,4,6.785281,4.896392,2.357921,1.642628
2,2,3394,5,11.012626,5.970491,3.067559,1.563465
3,3,3394,6,16.979798,10.966306,4.048612,2.897924
4,4,3394,7,28.943795,19.915152,4.034476,3.121276


In [43]:
df_mov_mean_std_pick_drop["pickups_mean"].max()


73.37120535714286

In [44]:
df_mov_mean_std_pick_drop_no_hour = df_mov_mean_std_pick_drop.groupby(["movement_id"]).mean().reset_index()[["movement_id","pickups_mean","dropoffs_mean","pickups_std","dropoffs_std"]]

In [45]:
df_mov_mean_std_pick_drop_no_hour.head()

,movement_id,pickups_mean,dropoffs_mean,pickups_std,dropoffs_std
0,3394,43.009629,33.976677,7.816031,6.776548
1,3411,9.408509,8.682742,4.734483,4.855788
2,3413,23.653786,18.884697,17.540844,13.834464
3,3414,17.068683,13.781300,13.784551,10.902070
4,3423,8.016456,7.820926,5.837589,5.015317


In [46]:
df_mov_mean_std_pick_drop_no_hour.shape

(128, 5)

In [47]:
df_mov_mean_std_pick_drop_no_hour.to_csv("../data/df_mov_mean_std_pick_drop_no_hour_sf_n_o_minimal.csv", index=False)

In [48]:
df_mov_mean_std_pick_drop_no_hour.to_json("../data/df_mov_mean_std_pick_drop_no_hour_sf_n_o_minimal.json")